In [13]:
import pandas as pd
import numpy as np 
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.manifold import TSNE

In [14]:
import xgboost 
xgboost.__version__

'0.90'

In [15]:
data = pd.read_csv('../Field_training_data_justMMETSP.csv')
labels = pd.read_csv('../Field_training_labels_justMMETSP.csv')
data = data.reset_index(drop=True)
labels = labels.reset_index(drop=True)

idx = labels.index[labels['Trophic mode'] == 'Un']
train_labels = labels.drop(idx)
train_data = data.drop(idx)

features = pd.read_csv('../Extracted_Pfams.csv')

testTranscriptomes_TPM_merged_lats = pd.read_csv('../../ForElaina/testTranscriptomes_old.csv')

In [16]:
testTranscriptomes_TPM_merged_lats.columns[(testTranscriptomes_TPM_merged_lats == 0).all()]

Index(['PF00821', 'PF01701', 'PF02419', 'PF03676', 'PF03762', 'PF06003',
       'PF06333', 'PF09631', 'PF09906', 'PF13427', 'PF14942', 'PF15001',
       'PF15113', 'PF15795', 'PF16414', 'PF16457', 'PF16670'],
      dtype='object')

In [17]:
features = features['Pfam'].str.split('.').str[0]

In [18]:
testTranscriptomes_TPM_merged_lats = testTranscriptomes_TPM_merged_lats[features]

In [19]:
testTranscriptomes_TPM_merged_lats.columns[(testTranscriptomes_TPM_merged_lats == 0).all()]

Index(['PF00821', 'PF01701', 'PF02419', 'PF03676', 'PF03762', 'PF06003',
       'PF06333', 'PF09631', 'PF09906', 'PF13427', 'PF14942', 'PF15001',
       'PF15113', 'PF15795', 'PF16414', 'PF16457', 'PF16670'],
      dtype='object')

In [20]:
train_data = train_data[features]

In [21]:
X,y = train_data, train_labels['Trophic mode']
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [22]:
from sklearn.model_selection import train_test_split
from sklearn import svm

In [23]:
#from https://scikit-learn.org/stable/modules/cross_validation.html
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [24]:
testTranscriptomes = scaler.transform(testTranscriptomes_TPM_merged_lats)

In [25]:
#model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
model = XGBClassifier(gamma=0.0, learning_rate=0.5, max_depth=3, n_estimators=10, reg_lambda=0.)
model.fit(X,y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.0,
              learning_rate=0.5, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=10, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=0.0, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [26]:
xg_predictions_testTranscriptomes = model.predict(testTranscriptomes)

In [27]:
prob = model.predict_proba(testTranscriptomes)

In [28]:
probDF = pd.DataFrame(data={'probability':np.max(prob, axis = 1)})

In [29]:
testTranscriptomes_predictions = pd.DataFrame(data={'xg_pred':xg_predictions_testTranscriptomes})

In [30]:
testTranscriptomes_predictions.to_csv('../../ForElaina/testTranscriptomes_predictions_old',index=False)

In [31]:
probDF.to_csv('../../ForElaina/testTranscriptomes_probabilities_old',index=False)